# Lesson 18: Research Loop with Query Generation and Human-in-the-Loop Feedback

In this lesson, we'll dive deep into the research loop that forms the heart of our research agent's intelligence. We'll explore how the agent generates relevant search queries based on content gathered in previous lessons and uses Perplexity to expand its knowledge base. We'll see also how to integrate human feedback into the research workflow. This creates a powerful human-in-the-loop system that allows users to guide the research direction while leveraging the agent's analytical capabilities.

Learning Objectives:
- Learn how to generate contextual research queries using LLMs and structured outputs
- Understand how to integrate external research services like Perplexity for comprehensive web search
- Implement human-in-the-loop feedback mechanisms in agentic workflows
- Explore the iterative design process behind building effective AI research agents

## 1. Setup

First, we define some standard Magic Python commands to autoreload Python packages whenever they change:

In [1]:
%load_ext autoreload
%autoreload 2

### Set Up Python Environment

To set up your Python virtual environment using `uv` and load it into the Notebook, follow the step-by-step instructions from the `Course Admin` lesson from the beginning of the course.

**TL/DR:** Be sure the correct kernel pointing to your `uv` virtual environment is selected.

### Configure Required APIs

To run this lesson, you'll need several API keys configured:

1. **Gemini API Key**, `GOOGLE_API_KEY` variable: Get your key from [Google AI Studio](https://aistudio.google.com/app/apikey).
2. **Perplexity API Key**, `PPLX_API_KEY` variable: Get your key from [Perplexity](https://www.perplexity.ai/settings/api).
3. **Firecrawl API Key**, `FIRECRAWL_API_KEY` variable: Get your key from [Firecrawl](https://firecrawl.dev/). They have a free tier that allows you to scrape 500 pages.
4. **Perplexity API Key**, `PPLX_API_KEY` variable: Get your key from [Perplexity](https://www.perplexity.ai/settings/api). This is required for the research loop functionality that performs web searches.

In [ ]:
from utils import env

env.load(required_env_vars=["GOOGLE_API_KEY", "PPLX_API_KEY", "FIRECRAWL_API_KEY", "PPLX_API_KEY"])

Environment variables loaded from `/Users/fabio/Desktop/course-ai-agents/.env`
Environment variables loaded successfully.


### Import Key Packages

In [3]:
import nest_asyncio
nest_asyncio.apply() # Allow nested async usage in notebooks

## 2. Understanding the Research Loop

The research loop is where our agent becomes truly intelligent. After gathering initial content from guidelines (as we saw in previous lessons), the agent enters a 3-round research cycle designed to fill knowledge gaps and expand understanding.

From the MCP prompt workflow, here's how the research loop works. We'll see its implementation in the next sections.

```markdown
3. Repeat the following research loop for 3 rounds:

    3.1. Run the "generate_next_queries" tool to analyze the ARTICLE_GUIDELINE_FILE, the already-scraped guideline
    URLs, and the existing PERPLEXITY_RESULTS_FILE. The tool identifies knowledge gaps, proposes new web-search
    questions, and writes them - together with a short justification for each - to the NEXT_QUERIES_FILE within
    NOVA_FOLDER.

    3.2. Run the "run_perplexity_research" tool with the new queries. This tool executes the queries with
    Perplexity and appends the results to the PERPLEXITY_RESULTS_FILE within NOVA_FOLDER.
```

## 3. The Philosophy: Plugging Into Specialized Services

Before diving into the implementation, it's important to understand our architectural philosophy. Similar to our approach with web scraping using Firecrawl, we're leveraging Perplexity for web search rather than building our own solution.

**Why use external services like Perplexity?**

Perplexity is an AI-powered search engine that combines the capabilities of large language models with real-time web search.

When there's a general problem faced by many in the industry, it's often more efficient to plug into dedicated tools rather than building every element yourself, similarly to what we did in the previous lesson with Firecrawl for scraping. Companies like Perplexity make LLM-based web search their entire business, investing heavily in:

- Comprehensive source coverage across the web
- Real-time information retrieval
- Advanced ranking and relevance algorithms
- Handling of dynamic content and paywalls
- Rate limiting and API reliability

This allows us to focus on the unique elements of our research agent: the intelligent query generation, human feedback integration, and workflow orchestration.

## 4. Query Generation: The Brain of the Research Loop

The `generate_next_queries` tool is where the magic happens. It analyzes all available context and intelligently identifies knowledge gaps to fill.

### 4.1 Understanding the Implementation

Let's examine the core implementation:

Source: _mcp_server/src/tools/generate_next_queries_tool.py_

```python
async def generate_next_queries_tool(research_directory: str, n_queries: int = 5) -> Dict[str, Any]:
    """
    Generate candidate web-search queries for the next research round.

    Analyzes the article guidelines, already-scraped content, and existing Perplexity
    results to identify knowledge gaps and propose new web-search questions.
    Each query includes a rationale explaining why it's important for the article.
    Results are saved to next_queries.md in the research directory.
    """
    # Convert to Path object
    research_path = Path(research_directory)
    nova_path = research_path / NOVA_FOLDER

    # Gather context from the research folder
    guidelines_path = research_path / ARTICLE_GUIDELINE_FILE
    results_path = nova_path / PERPLEXITY_RESULTS_FILE
    urls_from_guidelines_dir = nova_path / URLS_FROM_GUIDELINES_FOLDER

    article_guidelines = read_file_safe(guidelines_path)
    past_research = read_file_safe(results_path)

    # Collect all scraped content for context
    scraped_ctx_parts: List[str] = []
    if urls_from_guidelines_dir.exists():
        for md_file in sorted(urls_from_guidelines_dir.glob(f"*{MARKDOWN_EXTENSION}")):
            scraped_ctx_parts.append(md_file.read_text(encoding="utf-8"))
    scraped_ctx_str = "\n\n".join(scraped_ctx_parts)

    # Generate queries using LLM
    queries_and_reasons = await generate_queries_with_reasons(
        article_guidelines, past_research, scraped_ctx_str, n_queries=n_queries
    )

    # Write to next_queries.md (overwrite)
    next_q_path = nova_path / NEXT_QUERIES_FILE
    write_queries_to_file(next_q_path, queries_and_reasons)

    return {
        "status": "success",
        "queries_count": len(queries_and_reasons),
        "queries": queries_and_reasons,
        "output_path": str(next_q_path.resolve()),
        "message": f"Successfully generated {len(queries_and_reasons)} candidate queries..."
    }
```

The tool gathers three types of context:

1. **Article Guidelines** (`article_guidelines`): The original requirements and scope read from the `ARTICLE_GUIDELINE_FILE`. This provides the foundational understanding of what the article should cover.

2. **Past Research** (`past_research`): Previous Perplexity results read from the `PERPLEXITY_RESULTS_FILE`. This ensures the LLM doesn't generate duplicate queries and can identify gaps in existing research.

3. **Scraped Content** (`scraped_ctx_str`): All content from guideline URLs (web pages, GitHub repos, YouTube transcripts) concatenated together. This provides comprehensive background context that helps the LLM understand what information is already available.

The LLM analyzes all three contexts simultaneously. It uses the article guidelines to understand the target scope, reviews past research to see what's already been covered, and examines scraped content to understand the existing knowledge base. This comprehensive analysis enables it to generate queries that specifically target knowledge gaps. For example, if the article guidelines mention "error handling in AI agents" but neither past research nor scraped content covers this topic adequately, the LLM will prioritize generating queries about error handling strategies.

### 4.2 The LLM-Powered Query Generation

The actual query generation happens in the `generate_queries_with_reasons` function:

Source: _mcp_server/src/app/generate_queries_handler.py_

```python
async def generate_queries_with_reasons(
    article_guidelines: str,
    past_research: str,
    scraped_ctx: str,
    n_queries: int = 5,
) -> List[Tuple[str, str]]:
    """Return a list of tuples (query, reason)."""

    prompt = PROMPT_GENERATE_QUERIES_AND_REASONS.format(
        n_queries=n_queries,
        article_guidelines=article_guidelines or "<none>",
        past_research=past_research or "<none>",
        scraped_ctx=scraped_ctx or "<none>",
    )

    chat_llm = get_chat_model(settings.query_generation_model, GeneratedQueries)
    response = await chat_llm.ainvoke(prompt)

    queries_and_reasons = [(item.question, item.reason) for item in response.queries]
    return queries_and_reasons[:n_queries]
```

This is the prompt used for query generation. It can potentially have contexts of ~100k tokens or more:

Source: _mcp_server/src/config/prompts.py_

```python
PROMPT_GENERATE_QUERIES_AND_REASONS = """
You are a research assistant helping to craft an article.

Your task: propose {n_queries} diverse, insightful web-search questions
that, taken **as a group**, will collect authoritative sources for the
article **and** provide a short explanation of why each question is
important.

<article_guidelines>
{article_guidelines}
</article_guidelines>

<past_research>
{past_research}
</past_research>

<scraped_context>
{scraped_ctx}
</scraped_context>

Guidelines for the set of queries:
• Give priority to sections/topics from the article guidelines that
  currently lack supporting sources in <past_research> and
  <scraped_context>.
• Cover any remaining major sections to ensure balanced coverage.
• Avoid duplication; each query should target a distinct aspect.
• The web search queries should be natural language questions, not just keywords.
""".strip()
```

This prompt explicitly instructs the LLM to identify missing information and ensures queries cover different aspects rather than duplicating.

Last, the `generate_queries_with_reasons` function uses Pydantic models to ensure consistent, structured responses:

Source: _mcp_server/src/models/query_models.py_

```python
class QueryAndReason(BaseModel):
    """A single web-search query and the reason for it."""

    question: str = Field(description="The web-search question to research.")
    reason: str = Field(description="The reason why this question is important for the research.")

class GeneratedQueries(BaseModel):
    """A list of generated web-search queries and their reasons."""

    queries: List[QueryAndReason] = Field(description="A list of web-search queries and their reasons.")
```

This structured approach ensures the LLM returns exactly what we need: queries paired with clear justifications. These justifications have multiple purposes:
1. They help the LLM understand why each query is important for the research, similarly to how chain of thought prompts work.
2. They help us understand the LLM's thought process when generating queries, which is useful for debugging and improving the prompt. It's also useful for the human in the loop for providing useful feedback to the agent.

Let's test the `generate_queries_with_reasons` function to see how it works in practice:

In [ ]:
from research_agent_part_2.mcp_server.src.app.generate_queries_handler import generate_queries_with_reasons

# Example inputs (simplified for demonstration)
article_guidelines = '''
# Article: Advanced Function Calling with LLMs

## Sections to cover:
1. Error handling strategies
2. Performance optimization
3. Security considerations
4. Best practices for production
'''

past_research = '''
### Source [1]: https://example.com/basic-function-calling
Query: What is function calling in LLMs?
Answer: Function calling allows LLMs to invoke external tools and APIs...

### Source [2]: https://example.com/simple-examples  
Query: How to implement basic function calling?
Answer: Basic implementation involves defining function schemas...
'''

scraped_ctx = '''
# Function Calling Documentation
This guide covers the fundamentals of function calling...
[Basic examples and simple use cases already covered]
'''

# Generate queries based on this context
queries_and_reasons = await generate_queries_with_reasons(
    article_guidelines=article_guidelines,
    past_research=past_research,
    scraped_ctx=scraped_ctx,
    n_queries=3
)

for query, reason in queries_and_reasons:
    print(f"Query: {query}")
    print(f"Reason: {reason}")
    print("---")

/Users/fabio/Desktop/course-ai-agents/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "schema" in "FewShotExampleStructuredOutputCompliance" shadows an attribute in parent "BaseModel"
  warnings.warn(
/Users/fabio/Desktop/course-ai-agents/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Query: What are the most common security vulnerabilities when giving LLMs access to external tools via function calling, and what are the established mitigation strategies?
Reason: This question directly targets the 'Security considerations' section, a critical topic for production systems that is not covered in the existing research. It seeks to find authoritative sources on risks and specific, actionable solutions.
---
Query: How can latency be minimized and token usage be optimized when using chained or parallel function calls with large language models?
Reason: This addresses the 'Performance optimization' section. It focuses on advanced, practical challenges like speed and cost-efficiency, especially in complex scenarios that go beyond single, basic function calls.
---
Query: What are robust error handling patterns for LLM function calling when external API calls fail, return unexpected data, or the LLM hallucinates a function call?
Reason: This covers the 'Error handling strategi

2025-09-23 10:07:20.276 | ERROR    | logging:callHandlers:1762 | Research folder does not exist: /path/to/research_folder
2025-09-23 10:07:20.467 | INFO     | logging:callHandlers:1762 | Exception in execute request:
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[5], line 5
      3 # Update this path to your actual sample research folder
      4 research_folder = "/path/to/research_folder"
----> 5 result = await generate_next_queries_tool(research_directory=research_folder, n_queries=3)
      6 print(result)

File ~/Desktop/course-ai-agents/lessons/research_agent_part_2/mcp_server/src/tools/generate_next_queries_tool.py:76, in generate_next_queries_tool(research_directory, n_queries)
     73 nova_path = research_path / NOVA_FOLDER
     75 # Validate folders and files
---> 76 validate_research_folder(research_path)
     78 # Create NOVA_FOLDER directory if it doesn't exist
  

**Understanding the Output**

When you run this code, you'll see output similar to this example:

```
Query: What are the most common security vulnerabilities when implementing LLM function calling and how can they be mitigated?
Reason: This question directly targets the 'Security considerations' section of the article, which is not covered by existing research...

Query: What are effective strategies for optimizing the performance and reducing latency of function calling in large language models?
Reason: This question addresses the 'Performance optimization' section. The existing research covers basic implementation...

Query: What are the best practices for error handling and ensuring reliability in production-level LLM function calling systems?
Reason: This question is designed to gather information for the 'Error handling strategies' and 'Best practices for production' sections...
```

Observe the following:

1. **Gap Analysis**: Notice how the LLM identified that the existing research only covers "basic function calling" and "simple examples", then generated queries specifically targeting the missing advanced topics (security, performance, error handling).

2. **Section Mapping**: Each generated query directly maps to sections mentioned in the article guidelines that lack coverage. The LLM demonstrates sophisticated understanding by connecting article requirements to knowledge gaps.

3. **Query Quality**: The queries are well-crafted, specific, and actionable. Instead of generic questions like "What is security?", it asks targeted questions like "What are the most common security vulnerabilities when implementing LLM function calling?"

4. **Reasoning Transparency**: Each query comes with clear reasoning explaining why it's important and which article section it addresses. This transparency helps users understand the LLM's decision-making process.

5. **Progressive Complexity**: The LLM naturally progresses from basic concepts (already covered) to advanced topics (security, performance, production practices), showing an understanding of knowledge hierarchies.

This intelligent gap analysis is what makes the research loop so powerful - it doesn't just generate random queries, but strategically identifies exactly what information is missing to complete the article's objectives.


## 5. Perplexity Integration

Once we have our queries, we need to execute them efficiently. The `run_perplexity_research_tool` tool handles this integration with concurrent execution and structured outputs.

### 5.1 The Research Tool Implementation

The `run_perplexity_research_tool` is the orchestrator that manages the entire Perplexity research process:

Source: _mcp_server/src/tools/run_perplexity_research_tool.py_

```python
async def run_perplexity_research_tool(research_directory: str, queries: List[str]) -> Dict[str, Any]:
    """
    Run Perplexity research queries for the research folder.

    Executes the provided queries using Perplexity's Sonar-Pro model and appends
    the results to perplexity_results.md in the research directory. Each query
    result includes the answer and source citations.
    """
    research_path = Path(research_directory)
    nova_path = research_path / NOVA_FOLDER
    results_path = nova_path / PERPLEXITY_RESULTS_FILE

    if not queries:
        return {
            "status": "success",
            "message": "No queries provided – nothing to do.",
            "queries_processed": 0,
            "sources_added": 0,
        }

    # Execute all queries concurrently
    tasks = [run_perplexity_search(query) for query in queries]
    search_results = await asyncio.gather(*tasks)

    # Process and append search results to file
    total_sources = append_search_results_to_file(results_path, queries, search_results)

    return {
        "status": "success",
        "queries_processed": len(queries),
        "sources_added": total_sources,
        "output_path": str(results_path.resolve()),
        "message": f"Successfully completed Perplexity research round..."
    }
```

**How this function works:**

1. **Path Setup**: It establishes the file paths for storing results, specifically the `PERPLEXITY_RESULTS_FILE` in the `.nova` folder.

2. **Concurrent Execution of Perplexity Searches**: The key performance optimization is concurrent execution. Instead of running queries sequentially (which would be slow), it creates a list of tasks using `[run_perplexity_search(query) for query in queries]` and executes them all at once with `asyncio.gather(*tasks)`.

3. **Result Processing**: The `append_search_results_to_file` function processes all the search results and appends them to the persistent results file, maintaining unique source IDs across multiple research rounds.

4. **Cumulative Storage**: Results from each round are appended (not overwritten) to the same file, building up a comprehensive research database over the 3 rounds.

This design ensures fast execution while maintaining a complete audit trail of all research conducted.

### 5.2 Structured Perplexity Responses

The core Perplexity integration uses structured outputs to ensure consistent, parseable results. This is important: we're not just getting raw text responses, but structured data that our system can reliably process.

Source: _mcp_server/src/app/perplexity_handler.py_

```python
class SourceAnswer(BaseModel):
    """A single source answer with URL and content."""

    url: str = Field(description="The URL of the source")
    answer: str = Field(description="The detailed answer extracted from that source")

class PerplexityResponse(BaseModel):
    """Structured response from Perplexity search containing multiple sources."""

    sources: List[SourceAnswer] = Field(description="List of sources with their answers")

async def run_perplexity_search(query: str) -> Tuple[str, Dict[int, str], Dict[int, str]]:
    """Run a Perplexity Sonar-Pro search and return full answer + parsed sections."""

    # run perplexity search with structured output
    llm = get_chat_model("perplexity", PerplexityResponse)

    prompt = PROMPT_WEB_SEARCH.format(query=query)
    logger.debug(f"Searching web for: {query} …")

    response = await llm.ainvoke(prompt)

    # Convert structured response to the expected format
    answer_by_source = {}
    citations = {}
    for i, source in enumerate(response.sources, 1):
        answer_by_source[i] = source.answer
        citations[i] = source.url

    # Create a readable full answer string for compatibility
    full_answer_lines = []
    for i, source in enumerate(response.sources, 1):
        full_answer_lines.append(f"### [{i}]: {source.url}")
        full_answer_lines.append(source.answer)
        full_answer_lines.append("")
    full_answer = "\n".join(full_answer_lines)

    return full_answer, answer_by_source, citations
```

When you run a Perplexity search as above, you get a structured response like this:

```python
# Example of what run_perplexity_search returns for a query about "LLM function calling best practices"

full_answer = '''### [1]: https://openai.com/docs/guides/function-calling
Function calling allows models to connect to external tools and APIs...

### [2]: https://docs.anthropic.com/claude/docs/tool-use
Claude can use tools to perform actions beyond text generation...

### [3]: https://ai.google.dev/gemini-api/docs/function-calling
Gemini models support function calling for structured interactions...
'''

answer_by_source = {
    1: "Function calling allows models to connect to external tools and APIs. Best practices include defining clear schemas, handling errors gracefully, and validating inputs...",
    2: "Claude can use tools to perform actions beyond text generation. Key considerations include security, rate limiting, and proper error handling...",
    3: "Gemini models support function calling for structured interactions. Important aspects include schema design, response validation, and performance optimization..."
}

citations = {
    1: "https://openai.com/docs/guides/function-calling",
    2: "https://docs.anthropic.com/claude/docs/tool-use", 
    3: "https://ai.google.dev/gemini-api/docs/function-calling"
}
```

This structured format ensures that:
- **Each source is clearly separated** and can be processed independently
- **URLs are preserved** for citation and potential full scraping later
- **Content is substantial** (up to 300 words per source) for comprehensive coverage
- **Data is parseable** by downstream processing tools

### 5.3 The Perplexity Search Prompt

The prompt used for Perplexity searches is designed to extract maximum value:

```python
PROMPT_WEB_SEARCH = """
Question: {query}

Provide a detailed answer to the question above.
The answer should be organized into source sections, where each source section
contains all the information coming from a single source.
Never use multiple source citations in the same source section. A source section should refer to a single source.
Focus on the official sources and avoid personal opinions.
For each source, write as much information as possible coming from the source
and that is relevant to the question (at most 300 words).

Return a list of objects, where each object represents a source and has the following fields:
- url: The URL of the source
- answer: The detailed answer extracted from that source
""".strip()
```

This prompt ensures that each source is clearly identified and separated, that up to 300 words per source for detailed information, and that the output is in a consistent format for parsing and storage.

## 6 Testing the tools

### 6.1 Testing the Query Generation Tool

Let's test the query generation tool programmatically to understand its output:

In [8]:
from research_agent_part_2.mcp_server.src.tools import generate_next_queries_tool

# Update this path to your actual sample research folder
research_folder = "/path/to/research_folder"
result = await generate_next_queries_tool(research_directory=research_folder, n_queries=3)
print(result)

{'status': 'success', 'queries_count': 3, 'queries': [('What are the fundamental limitations of large language models that necessitate the use of external tools?', 'This question addresses the core "why" behind function calling, directly supporting the "Understanding why agents need tools" section of the article. The provided text explains *how* to use tools with a specific API, but this query will find sources that explain the foundational reasons, such as the inability to access real-time data, perform precise calculations, or interact with private APIs.'), ('How do the function calling or tool use APIs of major LLM providers like OpenAI, Gemini, and Anthropic compare in terms of features and implementation?', "The supplied context is exclusively about the Gemini API. To create a balanced and authoritative article, it's crucial to compare this with other major platforms. This query will uncover differences in syntax, capabilities (like parallel vs. sequential calling), and terminolog

The output will show a structured summary like:

```json
{
  "status": "success",
  "queries_count": 3,
  "queries": [
    ("What are the latest best practices for implementing function calling with LLMs?", "This query addresses implementation details that may not be fully covered in the basic documentation from the guidelines."),
    ("How do different LLM providers handle function calling differently?", "Understanding provider-specific approaches will help create more comprehensive guidance."),
    ("What are common pitfalls and debugging techniques for function calling?", "Practical troubleshooting information is essential for developers implementing these systems.")
  ],
  "output_path": "/path/to/research_folder/.nova/next_queries.md",
  "message": "Successfully generated 3 candidate queries for research folder..."
}
```

### 6.2 Testing the Perplexity Research Tool

Now let's test the Perplexity research functionality:

In [9]:
from research_agent_part_2.mcp_server.src.tools import run_perplexity_research_tool

# Example queries to test
test_queries = [
    "What are the latest developments in LLM function calling?",
    "How do you handle errors in AI agent tool execution?"
]

result = await run_perplexity_research_tool(
    research_directory=research_folder, 
    queries=test_queries
)
print("Tool call output:")
print(result)
print()

print("Content of the resulting file:")
with open(result["output_path"], "r") as f:
    print(f.read())

Tool call output:
{'status': 'success', 'queries_processed': 2, 'sources_added': 7, 'output_path': '/Users/fabio/Desktop/course-ai-agents/lessons/research_agent_part_2/data/sample_research_folder/.nova/perplexity_results.md', 'message': "Successfully completed Perplexity research round for research folder '/Users/fabio/Desktop/course-ai-agents/lessons/research_agent_part_2/data/sample_research_folder'. Processed 2 queries and added 7 source sections to perplexity_results.md"}

Content of the resulting file:
### Source [1]: https://arxiv.org/html/2505.20192v1

Query: What are the latest developments in LLM function calling?

Answer: The paper introduces **FunReason**, a novel framework designed to enhance large language models’ (LLMs) function calling capabilities. FunReason addresses the challenge of combining detailed reasoning with accurate function execution—a known limitation of prior approaches—by employing an automated data refinement strategy and a Self-Refinement Multiscale Los

## 7. Human-in-the-Loop

One of the most powerful aspects of our research agent is its ability to integrate human feedback directly into the workflow. This transforms the agent from a fully automated system into a collaborative research partner.

### 7.1 How Human Feedback Works

The agent can be instructed to pause after generating queries and ask for human approval before proceeding. This is accomplished by modifying the workflow instructions when triggering the MCP prompt.

When the user starts the research workflow, they can specify modifications like:
- "Ask for my feedback after generating each set of queries"
- "Show me the proposed queries and wait for my approval before running them"
- "Let me select which queries to run from the generated list"

Let's see how to run the complete research agent with human-in-the-loop feedback. We'll start the MCP client and demonstrate how to request user feedback integration.

In [10]:
from research_agent_part_2.mcp_client.src.client import main as client_main
import sys

async def run_client():
    _argv_backup = sys.argv[:]
    sys.argv = ["client"]
    try:
        await client_main()
    finally:
        sys.argv = _argv_backup

# Start client with in-memory server 
await run_client()

🛠️  Available tools: 11
📚 Available resources: 2
💬 Available prompts: 1

Available Commands: /tools, /resources, /prompts, /prompt/<name>, /resource/<uri>, /model-thinking-switch, /quit


🤔 LLM's Thoughts:
**Ready to Begin the Research Workflow!**

Okay, so I'm geared up and ready to dive into this research project. The user has given me the go-ahead, so let's get rolling. First things first, I need to outline the workflow. I'll make sure the user understands what I'm about to do, so they know what to expect.

Here's the plan, in a nutshell:

1.  **Setting the Stage (Setup):**  I'll start by taking a look at the `ARTICLE_GUIDELINE_FILE` to grab any URLs and local files that are relevant. It's like collecting my starting ingredients.

2.  **Processing the Resources (In Parallel):**  This is the bulk of the action.  I'll be working in parallel to speed things up. I'll be copying the local files, scraping and cleaning content from those websites, processing GitHub links (summarizing and c

I0000 00:00:1758615120.793915 3155870 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


🤔 LLM's Thoughts:
**Navigating the Research Workflow**

Okay, here's the deal. I've successfully completed the initial steps of this research project. I've extracted the guidelines' URLs (Step 1.3), checked for local files (there were none, Step 2.1), and scraped & cleaned other URLs (Step 2.2). Now, I need to execute Steps 2.3 and 2.4 concurrently - that's the next logical move, and it aligns perfectly with the requested workflow.

The real shift happens at Step 3, the research loop. The user wants me to be more interactive here. I have to pause after generating each set of research queries and get approval before running them through Perplexity. That's a key modification, and I'll need to implement that specifically.

Just to be clear, the research directory is `/Users/fabio/Desktop/course-ai-agents/lessons/research_agent_part_2/data/sample_research_folder`, and my task is to halt the process after Step 3. No need to move on to steps 4, 5, or 6. That's the objective. I'm ready to tac

Once the client is running, try these commands in sequence:

1. **Start the workflow with feedback**: `/prompt/full_research_instructions_prompt`

2. **Request human feedback integration**: When the agent asks for the research directory and workflow modifications, respond with:
   ```
   The research folder is /path/to/your/research_folder. Please modify the workflow to ask for my feedback after generating each set of queries in the research loop. Show me the proposed queries and wait for my approval before running them with Perplexity. Run up to step 3 of the workflow and then stop there, don't run the rest of the workflow from step 4 onwards.
   ```
   Otherwise, if you've run steps 1 and 2 of the workflow already, you can request the following:
   ```
   The research folder is /path/to/your/research_folder. Please modify the workflow to ask for my feedback after generating each set of queries in the research loop. Show me the proposed queries and wait for my approval before running them with Perplexity. Run only step 3 of the workflow and then stop there, don't run the rest of the workflow from step 4 onwards. Don't run steps 1 and 2 of the workflow.
   ```
3. **Observe the agent behavior**: The agent will:
   - Run the initial data ingestion (steps 1-2)
   - Generate the first set of research queries
   - **Pause and show you the queries**
   - Wait for your feedback before proceeding, allowing you to approve, modify, or reject queries

4. **Provide feedback**: You can respond with:
   - "Approve all queries" to proceed as planned
   - "Only run queries 1, 3, and 5" to select specific queries
   - "Replace query 2 with: [your custom query]" to modify the research direction

5. **Continue the loop**: The agent will repeat this process for each of the 3 research rounds. You can stop the loop by responding with `/quit"`.